In [ ]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

model = ChatOpenAI(temperature = 0)

messages = [
]
from langchain.prompts import ChatPromptTemplate


template = ChatPromptTemplate.from_messages([
    ("system", " you are an intelligent agent. answer user's prompt accurately"),
    ("user", "{prompt}"),
    ("system", f"history for context: {messages}") # this does not work (as its not dynamic)
    
])

chain = template | model


In [ ]:
from typing import List
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from pydantic import BaseModel, Field


class Add(BaseModel):
    '''
        add numbers
    '''
    numbers: List[int] = Field(description="list containing integer numbers to add")
    

tools = [convert_pydantic_to_openai_function(Add)]


model.bind(functions = tools)
# note it returns a new model with functions


model_with_tools = model.bind(functions = tools)

template = ChatPromptTemplate.from_messages([
    ("act like a dumb person who doesn't know how to perform even the basic operations like addition!"),
    ("user", "{input}")
])

chain = template | model_with_tools

chain.invoke({"input": "can you add 1 and 2 please"})


'''
o/p:

chain.invoke({"input": "can you add 1 and 2 please"})
AIMessage(content='', additional_kwargs={'function_call': {'name': 'Add', 'arguments': '{"numbers":[1,2]}'}})
'''